In [1]:
# 5.1. Layers and Blocks
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)

tensor([[-0.2398, -0.0877,  0.0924, -0.2416,  0.0796,  0.0471, -0.1513, -0.2583,
          0.1561,  0.1780],
        [-0.1019, -0.1117, -0.0371, -0.2143,  0.0813, -0.0046, -0.1364, -0.0924,
          0.1080,  0.1188]], grad_fn=<AddmmBackward>)

In [2]:
# 5.1.1. A Custom Block
class MLP(nn.Module):
    # Declare a layer with model parameters. Here, we declare two fully
    # connected layers
    def __init__(self):
        # Call the constructor of the `MLP` parent class `Module` to perform
        # the necessary initialization. In this way, other function arguments
        # can also be specified during class instantiation, such as the model
        # parameters, `params` (to be described later)
        super().__init__()
        self.hidden = nn.Linear(20, 256)  # Hidden layer
        self.out = nn.Linear(256, 10)  # Output layer

    # Define the forward propagation of the model, that is, how to return the
    # required model output based on the input `X`
    def forward(self, X):
        # Note here we use the funtional version of ReLU defined in the
        # nn.functional module.
        return self.out(F.relu(self.hidden(X)))

In [3]:
net = MLP()
net(X)

tensor([[-0.0656,  0.0463,  0.0144, -0.0097,  0.0605,  0.1886, -0.1313, -0.1102,
         -0.1371, -0.2995],
        [-0.1384,  0.0333, -0.0109,  0.0135,  0.0311,  0.2013, -0.2052, -0.0649,
         -0.1348, -0.2761]], grad_fn=<AddmmBackward>)

In [4]:
# 5.1.2. The Sequential Block
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            # Here, `module` is an instance of a `Module` subclass. We save it
            # in the member variable `_modules` of the `Module` class, and its
            # type is OrderedDict
            self._modules[str(idx)] = module

    def forward(self, X):
        # OrderedDict guarantees that members will be traversed in the order
        # they were added
        for block in self._modules.values():
            X = block(X)
        return X

In [5]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[ 0.1787, -0.0067,  0.3157,  0.0217,  0.1676,  0.1165,  0.0692,  0.0396,
         -0.1890,  0.1298],
        [ 0.2467, -0.0819,  0.3407,  0.0273,  0.0777,  0.1568,  0.0973,  0.0700,
         -0.1124,  0.0732]], grad_fn=<AddmmBackward>)

In [6]:
# 5.1.3. Executing Code in the Forward Propagation Function
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # Random weight parameters that will not compute gradients and
        # therefore keep constant during training
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        # Use the created constant parameters, as well as the `relu` and `mm`
        # functions
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        # Reuse the fully-connected layer. This is equivalent to sharing
        # parameters with two fully-connected layers
        X = self.linear(X)
        # Control flow
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

In [7]:
net = FixedHiddenMLP()
net(X)

tensor(0.1356, grad_fn=<SumBackward0>)

In [8]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(0.0365, grad_fn=<SumBackward0>)